In [1]:
from client import Client
import requests
import json
import numpy as np
from sklearn.metrics import log_loss


client = Client("node2","0.0.0.0")

In [2]:
dataset = client.load_data("../datasets/UNSW_NB15_training-set.csv")

In [3]:
x, y = client.preprocess_data(dataset)

In [4]:
y_test = y[60000:]
x_test = x[60000:]

In [5]:
x = x[30000:60000]
y = y[30000:60000]

In [6]:
coef, intercept, acc, clf = client.train_model(x,y)

/home/tk/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
client.test_model(clf,y_test,x_test)

0.25497044599677593

In [8]:
loss = log_loss(y_test,clf.predict(x_test), eps=1e-15)
print(loss)

25.73299705573752


In [36]:
headers = {'content-type': 'application/json'}
url = 'http://127.0.0.1:5000/register'

data = {
    "username" : "Sam",
    "email" : "sam@test.com",
    "password" : "123456789"
}

response = requests.post(url, data=json.dumps(data), headers=headers)

In [37]:
response.text

'{"id": 2}'

In [38]:
headers = {'content-type': 'application/json'}
url = 'http://127.0.0.1:5000/local_model'

data = {
    "intercept" : intercept[0],
    "bias" : coef.tolist()[0],
    "author" : 2
}

response = requests.post(url, data=json.dumps(data), headers=headers)
response.text

'{"result": "model added"}'

### After global model calculation

In [47]:
headers = {'content-type': 'application/json'}
url = 'http://127.0.0.1:5000/model/global'

x = requests.get(url, headers=headers)
json = x.json()
coef = json['bias']
inter = json['intercept']

In [48]:
clf.intercept_ = np.array([inter])
clf.coef_ = np.array([coef])

In [49]:
client.test_model(clf,y_test,x_test)

0.2547465520329572